In [10]:
import casadi as ca
import numpy as np
import sys

sys.path.insert(0, '../python/pyecca')

import pyecca.lie.so3 as so3

def aero_prop():
    pos_ENU = ca.SX.sym('pos_ENU', 3)
    vel_ENU = ca.SX.sym('vel_ENU', 3)
    q_ENU_TRF = ca.SX.sym('q_ENU_TRF', 4)
    p = ca.SX.sym('p', 15)
    u = ca.SX.sym('u', 4)

    C_NED_ENU = np.array([
        [0, 1, 0],
        [1, 0, 0],
        [0, 0, -1]
    ])

    C_TRF_FRD = np.array([
        [0, 0, -1],
        [0, 1, 0],
        [1, 0, 0]
    ])
    
    r_NED_FRD = so3.Mrp.from_dcm(
        ca.mtimes([C_NED_ENU, so3.Dcm.from_quat(q_ENU_TRF), C_TRF_FRD]))
    
    F_aero = ca.vertcat(0, 0, 0)
    F_prop = ca.vertcat(0, 0, 0)

    f = ca.Function('aero_prop', [pos_ENU, vel_ENU, q_ENU_TRF], [F_aero, F_prop],
        ['pos_ENU', 'vel_ENU', 'q_ENU_TRF'], ['F_aero', 'F_prop'])

    gen = ca.CodeGenerator('casadi_gen.c', {'main': False, 'mex': False, 'with_header': True, 'with_mem': True})
    gen.add(f)
    gen.generate()

aero_prop()

In [11]:
dir(pyecca)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_version',
 'lie']